In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec

In [7]:
#Tiền xử lý dữ liệu
def tokenize_vietnamese_text(text):
    return text.split()

def filter_not_alphabet_or_number(text):
    return re.sub('\W+', ' ', text)

# Đọc dữ liệu từ file CSV
file_path = "/content/drive/MyDrive/chatbot/clustered.csv"  # Thay "path_to_your_csv_file.csv" bằng đường dẫn tới file CSV thực tế
df = pd.read_csv(file_path)

# Lấy dữ liệu từ cột có nhãn là "Question"
text_column = df["Question"]

# Tách từng dòng của cột "Question"
x_data = []
for text in text_column:
    text = filter_not_alphabet_or_number(text)
    x_data.append(text)

print(x_data)

['em cám ơn ad nhiều ạ', 'yêu add quá đi', 'vâng e cám ơn nhé', 'dạ em cám ơn ạ ', 'vâng em cám ơn ạ', 'thông tin tuyệt vời ạ', 'à oke bạn', 'da vâng em cảm ơn ạ ', 'da vâng ạ', 'da cô', 'da tl e vs ạ', 'à vâng ạ', 'à vâng', 'vâng ạ ', 'vâng đúng r ạ', 'ạ vâng', 'vâng ạ ', 'dạ vâng ạ', 'dạ vâng ad', 'dạ r ạ', 'dạ dạ ', 'dạ đc ạ', 'à dạ', 'dạ vâng ạ ', 'dạ vâng', 'dạ vâng ạ ', 'dạ vâng', 'dạ vâng ạ', 'dạ vâng e nhận đc r ạ', 'ok chị', 'ok ad', 'ok ad ad ơi', 'ok anh chị', 'dạ ok em cảm ơn ad', 'ok em cam ơn nha ', 'ok vâng ạ', 'ok ạ vậy đắng kí lại từ đầu ạ', 'xong nộp học bạ là ok à a', 'ok ngon ngay', 'ok cô', 'dạ ok ad', 'vang c e cảm on', 'vang em cam on a', 'vang a', 'vang ạ', 'vang a còn bọn em ạ', 'chị ơi', 'ad ơi cho em', 'dạ ad ơi 3', 'ad ơi ', 'ad ơi e hỏi', 'ad ơi em hỏi', 'vg ah', 'vg ', 'vg e cảm ơn ạ', 'vg em cảm ơn', 'vg ạ', 'vg e cảm ad nhiều ạ', 'vg ạ tks ad', 'vâg ạ', 'vâg em cảm ơn ạ', 'dạ vâg', 'vâg e cảm ơn ạ', 'vâg ạk', 'dạ vâg ạ', 'à vâg em cảm ơn', 'vâg em cảm ơn

In [8]:
# Đánh nhãn
y_data = df["lable"]

labeling_answer = {}
time_appear = {}
labeling_count = 1
new_y_data = []
for intent in y_data:
    if intent not in labeling_answer:
      labeling_answer[intent] = labeling_count
      labeling_count += 1
      time_appear[intent] = 0
    else:
      time_appear[intent] += 1
    new_y_data.append(labeling_answer[intent])

print(new_y_data)
print('labeling_answer: {}'.format(labeling_answer))
print(time_appear)
print(len(new_y_data))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [9]:
input_data = x_data
output_data = new_y_data

print(x_data[:-5])
print(output_data[:-5])

# Encoding responses into numerical labels
labels = new_y_data

# Data preprocessing: TF-IDF vectorization
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(input_data)

# Word embeddings using Word2Vec
sentences = [sent.split() for sent in input_data]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

# Create word embeddings for each sentence
X_w2v = []
for sent in sentences:
    embeddings = [word2vec_model.wv[word] for word in sent if word in word2vec_model.wv]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        X_w2v.append(avg_embedding)
    else:
        # If a sentence contains no known words, add a zero vector
        X_w2v.append(np.zeros(word2vec_model.vector_size))

X_w2v = np.array(X_w2v)

# Combine TF-IDF and Word2Vec features
X_combined = np.hstack((X_tfidf.toarray(), X_w2v))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, labels, test_size=0.2, random_state=42)

# Train SVM model
svm_model = SVC(C=10.0, kernel='rbf', gamma=0.1, coef0=0.0)
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

['em cám ơn ad nhiều ạ', 'yêu add quá đi', 'vâng e cám ơn nhé', 'dạ em cám ơn ạ ', 'vâng em cám ơn ạ', 'thông tin tuyệt vời ạ', 'à oke bạn', 'da vâng em cảm ơn ạ ', 'da vâng ạ', 'da cô', 'da tl e vs ạ', 'à vâng ạ', 'à vâng', 'vâng ạ ', 'vâng đúng r ạ', 'ạ vâng', 'vâng ạ ', 'dạ vâng ạ', 'dạ vâng ad', 'dạ r ạ', 'dạ dạ ', 'dạ đc ạ', 'à dạ', 'dạ vâng ạ ', 'dạ vâng', 'dạ vâng ạ ', 'dạ vâng', 'dạ vâng ạ', 'dạ vâng e nhận đc r ạ', 'ok chị', 'ok ad', 'ok ad ad ơi', 'ok anh chị', 'dạ ok em cảm ơn ad', 'ok em cam ơn nha ', 'ok vâng ạ', 'ok ạ vậy đắng kí lại từ đầu ạ', 'xong nộp học bạ là ok à a', 'ok ngon ngay', 'ok cô', 'dạ ok ad', 'vang c e cảm on', 'vang em cam on a', 'vang a', 'vang ạ', 'vang a còn bọn em ạ', 'chị ơi', 'ad ơi cho em', 'dạ ad ơi 3', 'ad ơi ', 'ad ơi e hỏi', 'ad ơi em hỏi', 'vg ah', 'vg ', 'vg e cảm ơn ạ', 'vg em cảm ơn', 'vg ạ', 'vg e cảm ad nhiều ạ', 'vg ạ tks ad', 'vâg ạ', 'vâg em cảm ơn ạ', 'dạ vâg', 'vâg e cảm ơn ạ', 'vâg ạk', 'dạ vâg ạ', 'à vâg em cảm ơn', 'vâg em cảm ơn

In [24]:
# Get answer by id
def getAnswer(id):
  keys = [k for k, v in labeling_answer.items() if v == id]
  if len(keys) == 1:
    answer = df.loc[df['lable'] == keys[0]]
    return answer.iloc[0, 1]
  else:
    return None

# Chatbot function
def chatbot(query):
    query_vectorized_tfidf = vectorizer.transform([query])
    query_vectorized_w2v = []
    for word in query.split():
        if word in word2vec_model.wv:
            query_vectorized_w2v.append(word2vec_model.wv[word])
    if query_vectorized_w2v:
        query_vectorized_w2v = np.mean(query_vectorized_w2v, axis=0)
    else:
        # If no known words in the query, use a zero vector
        query_vectorized_w2v = np.zeros(word2vec_model.vector_size)

    query_vectorized_combined = np.hstack((query_vectorized_tfidf.toarray(), query_vectorized_w2v.reshape(1, -1)))
    response_label = svm_model.predict(query_vectorized_combined)[0]
    response = getAnswer(response_label)
    return response

# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    response = chatbot(user_input)
    print("Chatbot:", response)

You: chào
Chatbot: Chào em, em cần gì?
You: nào nộp hồ sơ vậy
Chatbot: Em có thể đến trực tiếp các cơ sở của trường nhé Cơ sở khoái châu: trung tâm tuyển sinh & truyền thông, xã dân tiến, khoái châu, hưng yên Cơ sở mỹ hào: tx mỹ hào, mỹ hào hưng yên Cơ sở hải dương: 189 nguyễn lương bằng, tp hải dương 
You: hạn chót nộp hồ sơ á cô
Chatbot: Đợt 1: từ tháng 5/2020 đến 30/6/2020. dự kiến ngày 30/6/2020 nhà trường sẽ công bố kết quả thí sinh đủ điều kiện sơ tuyển vào học tại trường, giấy báo nhập học được cấp khi thí sinh được công nhận tốt nghiệp thpt. Nhà trường nhận hồ sơ và xét tuyển liên tục em nhé, em nộp hồ sơ sớm để tỷ lệ đỗ chắc chắn hơn nhé Em theo dõi fanpage trường để cập nhật thông tin mới nhất nhé: https://www.facebook.com/daihocsuphamkythuathungyen/?ref=bookmarks  


KeyboardInterrupt: ignored